In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X, Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:2000])

WITH "DT_node_lookup" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_2" <= 2.450000047683716) THEN 1 ELSE CASE WHEN ("ADS"."Feature_3" <= 1.75) THEN CASE WHEN ("ADS"."Feature_2" <= 4.949999809265137) THEN CASE WHEN ("ADS"."Feature_3" <= 1.6500000953674316) THEN 5 ELSE 6 END ELSE CASE WHEN ("ADS"."Feature_3" <= 1.5499999523162842) THEN 8 ELSE CASE WHEN ("ADS"."Feature_2" <= 5.449999809265137) THEN 10 ELSE 11 END END END ELSE CASE WHEN ("ADS"."Feature_2" <= 4.850000381469727) THEN CASE WHEN ("ADS"."Feature_1" <= 3.0999999046325684) THEN 14 ELSE 15 END ELSE 16 END END END AS node_id_2 
FROM "INPUT_DATA" AS "ADS"), 
"DT_node_data" AS 
(SELECT "Values".node_id AS node_id, "Values".feature AS feature, "Values".threshold AS threshold, "Values".count AS count, "Values".depth AS depth, "Values".parent_id AS parent_id, "Values"."Proba_0" AS "Proba_0", "Values"."LogProba_0" AS "LogProba_0", "Values"."Proba_1" AS "Proba_1", "Values"."LogProba_1" AS "LogProba_1", "Values"."Proba_2" AS 

## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);


In [9]:
sql_output.sample(12)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
97,97,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
96,96,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
136,136,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
118,118,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
72,72,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
26,26,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
83,83,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
102,102,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
74,74,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
64,64,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1


In [10]:
sql_output.Decision.value_counts()

2    50
1    50
0    50
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12)


/usr/lib/python3/dist-packages/sklearn/tree/tree.py:791: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
29,29,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,0
128,128,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,2
124,124,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,2
39,39,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,0
45,45,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,0
10,10,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,0
30,30,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,0
63,63,NaN,NaN,NaN,0.0,1.0,0.0,-inf,0.000000,-inf,1
67,67,NaN,NaN,NaN,0.0,1.0,0.0,-inf,0.000000,-inf,1
112,112,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,2


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
114,114,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
106,106,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
134,134,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
113,113,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
54,54,NaN,NaN,NaN,0.0,1.0,0.0,-inf,0.000000,-inf,...,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
78,78,NaN,NaN,NaN,0.0,1.0,0.0,-inf,0.000000,-inf,...,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
40,40,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,...,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
18,18,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,...,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
107,107,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
102,102,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
